In [1]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
import common as com
from torch.utils.data import DataLoader
from IPython.display import Markdown, display
from rouge_score import rouge_scorer

In [2]:
def printmd(string):
    display(Markdown(string))

In [3]:
model_name = "31_model_w_prefix_50_output.pt"
model_finetuned = torch.load(f"models/{model_name}")
model_finetuned.eval();
tokenizer = T5Tokenizer.from_pretrained("t5-base", model_max_length=com.MAX_INPUT_LEN)
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

model_bare = T5ForConditionalGeneration.from_pretrained("t5-base")
model_bare.eval();

In [4]:
def summarize_one_text(text: str, model) -> str:
    text = "summarize: " + text
    encoded = tokenizer(text, max_length=com.MAX_INPUT_LEN, pad_to_max_length=True,
                                       truncation=True,
                                       padding="max_length", return_tensors="pt")
    pred_ids = model.generate(
        input_ids=encoded.input_ids,
        attention_mask=encoded.attention_mask,
        max_length=com.MAX_OUTPUT_LEN)
    pred_text = tokenizer.decode(pred_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return pred_text

In [31]:
_, test_data = com.load_and_maybe_split_data(com.TRAIN_FRACTION, com.RANDOM_SEED)

In [14]:
predictions_tuned = []
predictions_bare = []
targets = []
texts = []
for i, row in test_data.iterrows():
    review = row['review']
    summary = row['summary']
    
    fine_tuned_summary = summarize_one_text(review, model_finetuned)
    bare_summary = summarize_one_text(review, model_bare)

    predictions_tuned.append(fine_tuned_summary)
    predictions_bare.append(bare_summary)
    targets.append(summary)
    texts.append(review)

In [17]:
index_list = iter(range(len(targets)))

In [18]:
len(predictions_tuned) == len(predictions_bare) == len(targets) == len(texts)

True

In [26]:
i = next(index_list)

target_sum = targets[i]
generated_sum = predictions_tuned[i]
review = texts[i]
bare_pred = predictions_bare[i]

len_original = len(review.split(' '))
len_tar_summary = len(target_sum.split(' '))
len_gen_summary = len(generated_sum.split(' '))
len_gen_summary_bare = len(bare_pred.split(' '))
scores = scorer.score(target_sum, generated_sum)

printmd(f"**Original review**: {review} (**{len_original}**)\n")
printmd(f"**Target summary**: {target_sum} (**{len_tar_summary}**)\n")
printmd(f"**Gererated summary**:\n {generated_sum} (**{len_gen_summary}**)\n")
printmd(f"**Gererated summary from 'bare' T5**:\n {bare_pred} (**{len_gen_summary_bare}**)\n")
# printmd("Rouge1 F1 score: {0:.2f}\n".format(scores['rouge1'].fmeasure))
# printmd("RougeL F1 score: {0:.2f}\n".format(scores['rougeL'].fmeasure))



**Original review**: This is the first cordless hoover I have had and it is lovely not having the wire get in the way and the constant moving of the plug. The charge is ok it's enough to do the whole house. We have got in the habit of charging straight after use or it can be frustrating if you forget and go to Hoover and the batter is dead. It takes a few hours to charge. I have had a couple of issues one with the charger and one with the hoover but I contacted Planet Express and they were extremely helpful and sent replacement parts. This is really important for me knowing if there are problems the company will resolve it and someone will reply to my contact. I would recommend this hoover (**132**)


**Target summary**: The cordless vacuum provides freedom from wires and has a sufficient charge for the whole house. Charging immediately after use is necessary to avoid frustration. The company's excellent customer service resolves any issues promptly. Overall, highly recommended. (**37**)


**Gererated summary**:
 The cordless hoover is lovely not having the wire get in the way and the constant moving of the plug. The charge is ok it's enough to do the whole house. It takes a few hours to charge. The company has been extremely helpful and sent replacement parts. I would recommend this hoover. (**53**)


**Gererated summary from 'bare' T5**:
 the hoover takes a few hours to charge and is lovely not having the wire get in the way. the charger is ok it's enough to do the whole house. (**30**)
